In [ ]:
import torch
import time
import threading
import queue

def worker_task(matrix_size, duration, result_queue, cuda=False):
    """Задача для каждого потока"""
    local_iterations = 0
    start_time = time.time()
    
    while time.time() - start_time < duration:
        a = torch.randn(matrix_size, matrix_size)
        b = torch.randn(matrix_size, matrix_size)
        
        if cuda:
            a = a.cuda()
            b = b.cuda()

        # Интенсивные вычисления
        for _ in range(10):
            c = torch.matmul(a, b)
            a = torch.matmul(c, a.t())
        
        local_iterations += 1
    
    result_queue.put(local_iterations)

def cpu_multithread_stress_test(duration_sec=30, num_threads=8, matrix_size=1024, cuda=False):
    """Многопоточный стресс-тест CPU"""
    print(f"Запуск {num_threads} потоков...")
    
    threads = []
    result_queue = queue.Queue()
    
    # Запуск потоков
    for i in range(num_threads):
        t = threading.Thread(
            target=worker_task,
            args=(matrix_size, duration_sec, result_queue, cuda)
        )
        threads.append(t)
        t.start()
        print(f"Поток {i+1} запущен")
    
    # Ожидание завершения
    for t in threads:
        t.join()
    
    # Сбор результатов
    total_iterations = 0
    while not result_queue.empty():
        total_iterations += result_queue.get()
    
    print(f"\nВсего выполнено операций: {total_iterations}")
    print(f"Средняя скорость: {total_iterations/duration_sec:.2f} опер/сек")

# Тест с 4 потоками на 30 секунд
cpu_multithread_stress_test(duration_sec=60, num_threads=16, cuda=False)

Запуск 16 потоков...
Поток 1 запущен
Поток 2 запущен
Поток 3 запущен
Поток 4 запущен
Поток 5 запущен
Поток 6 запущен
Поток 7 запущен
Поток 8 запущен
Поток 9 запущен
Поток 10 запущен
Поток 11 запущен
Поток 12 запущен
Поток 13 запущен
Поток 14 запущен
Поток 15 запущен
Поток 16 запущен


In [ ]:
# всего выполнено операций: 1319
# Средняя скорость: 22.0 опер/сек

In [ ]:
x = torch.randn(10**6).cuda()

In [ ]:
x - 2

tensor([ 0.1832,  1.0823, -1.1020, -2.2206, -0.6066, -0.6003, -1.8771, -1.5581,
        -0.5125,  0.0037], device='cuda:0')

In [ ]:
x - 3

tensor([-1.8168, -0.9177, -3.1020, -4.2206, -2.6066, -2.6003, -3.8771, -3.5581,
        -2.5125, -1.9963], device='cuda:0')

In [ ]:
x - 4

tensor([-2.8168, -1.9177, -4.1020, -5.2206, -3.6066, -3.6003, -4.8771, -4.5581,
        -3.5125, -2.9963], device='cuda:0')